In [ ]:
### Drive Access
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
### Importing the necessary libraries
import warnings
import sys
import io
import pandas as pd 
import numpy as np 
import matplotlib.pyplot as plt
import seaborn as sns
import pickle
from matplotlib import pyplot
from sklearn import datasets
from sklearn.model_selection import train_test_split 
from sklearn.model_selection import GridSearchCV
from sklearn.model_selection import KFold
from sklearn.linear_model import Lasso,Ridge
from sklearn.decomposition import PCA
import matplotlib.pyplot as plt
from sklearn.preprocessing import StandardScaler
from sklearn.preprocessing import MinMaxScaler
from sklearn.linear_model import LinearRegression
from sklearn.metrics import r2_score
from sklearn.metrics import mean_squared_error
from google.colab import drive
from google.colab import files
from collections import *
from math import sqrt,ceil,log2,floor,pi
from datetime import datetime
from copy import deepcopy

warnings.filterwarnings("ignore")

### Reading the csv file
np.random.seed(0)
notebookpath = "/content/drive/MyDrive/Colab Notebooks/SanityCheck.csv"
df = pd.read_csv(notebookpath)
#df

In [ ]:
### Preprocessing for MLR
df1 = deepcopy(df)
df1 = df1.drop(columns = ['Symbol','Series','Volume','VWAP','Deliverable Volume','%Deliverble'], axis = 1)
lastcol = len(df1.columns)
dfnum = df1[df1.columns[list(range(2,lastcol))]]
cols = list(dfnum.columns)
meantrades = df1['Trades'].mean()
df1['Trades'].fillna(value=meantrades, inplace=True)
data1 = pd.DataFrame(df1, columns = cols)
targetfeature = 'Close'

### Preprocessing for LSTM
df2 = deepcopy(df)
meantrades = df2['Trades'].mean()
df2['Trades'].fillna(value=meantrades, inplace=True)
df2 = df2.drop(columns = ['Symbol','Series','Volume','VWAP','Deliverable Volume','%Deliverble'], axis = 1)
lastcol = len(df2.columns)
dfstr = df2[df2.columns[list(range(0,2))]]
dfnum = df2[df2.columns[list(range(2,lastcol))]]
scaler = MinMaxScaler()
cols = list(dfnum.columns)
dfnum = scaler.fit_transform(dfnum)
smin = scaler.data_min_
smax = scaler.data_max_
dfnum = pd.DataFrame(dfnum, columns = cols)
dfscaled = pd.concat([dfstr , dfnum], axis = 1, join = 'inner')
dfscaled = dfnum
data2 = deepcopy(dfscaled)

### Preprocessing for ElasticNet
df3 = deepcopy(df)
meantrades = df3['Trades'].mean()
df3['Trades'].fillna(value=meantrades, inplace=True)
df3 = df3.drop(columns = ['Symbol','Series'], axis = 1)
lastcol = len(df3.columns)
dfstr = df3[df3.columns[list(range(0,2))]]
dfnum = df3[df3.columns[list(range(2,lastcol))]]
cols = list(dfnum.columns)
dfnum = pd.DataFrame(dfnum, columns = cols)  
dfscaled = pd.concat([dfstr , dfnum], axis = 1, join = 'inner')
dfscaled = dfnum
data3 = deepcopy(dfscaled)

### Preprocessing for CNN
df4 = deepcopy(df)
meantrades = df4['Trades'].mean()
df4['Trades'].fillna(value=meantrades, inplace=True)
df4 = df4.drop(columns = ['Symbol','Series','Volume','VWAP','Deliverable Volume','%Deliverble'], axis = 1)
data4 = deepcopy(df4)

In [ ]:
### Window Size is 5

### MLR + Ridge
window = 5

j = 0;
def transform(dfscaled, win):
  df2 = deepcopy(dfscaled)
  col = df2.columns
  for i in range(2, win+1):
    for c in col:
      df2[c+str(i)] = df2[c].shift(-(i-1)) #stepback i-1 times
      df2['Close'+str(win+1)] = df2['Close'].shift(-(win)) #stepback i-1 times
  return df2
print("window"+str(window))
data = transform(data1, window)
scaler = MinMaxScaler()
data1 = scaler.fit_transform(data)
data1 = pd.DataFrame(data1)
   
MLR = pickle.load(open('MLR_Ridge_5.pkl', 'rb'))
trainn = deepcopy(data.loc[0:10])
testt = deepcopy(data.loc[10:15])
trainnY = trainn["Close"+str(window+1)].values
del trainn["Close"+str(window+1)]
trainn.drop(columns=[])
trainnX = trainn.values.astype(np.float64)
testtY = testt["Close"+str(window+1)].values
del testt["Close"+str(window+1)]
testtX = testt.values.astype(np.float64) 
testp = MLR.predict(testtX)
print(testp)
print(mean_squared_error(testp, testtY))

In [ ]:
### Window Size is 5

### LSTM
window = [5]

def transdata(dfscaled, win):
  df2 = deepcopy(dfscaled)
  col = df2.columns
  for i in range(2, win+1):
    for c in col:
      df2[c+str(i)] = df2[c].shift(-(i-1)) #stepback i-1 times
  
  df2['Close'+str(win+1)] = df2['Close'].shift(-(win)) #stepback i-1 times
  return df2

data = transdata(data2, 5)
LSTM = pickle.load(open('model.pkl', 'rb'))
trainn = deepcopy(data.loc[0:10])
testt = deepcopy(data.loc[10:15])
trainnY = trainn["Close"+str(window+1)].values
del trainn["Close"+str(window+1)]
trainn.drop(columns=[])
trainnX = trainn.values.astype(np.float64)
testtY = testt["Close"+str(window+1)].values
del testt["Close"+str(window+1)]
testtX = testt.values.astype(np.float64) 
testp = LSTM.predict(testtX)
print(mean_squared_error(testp, testtY))

In [ ]:
### Window Size is 5

### Elastic Net
window = [5]

def transdata(dfscaled, win):
  df2 = deepcopy(dfscaled)
  col = df2.columns
  for i in range(2, win+1):
    for c in col:
      df2[c+str(i)] = df2[c].shift(-(i-1)) #stepback i-1 times
  
  df2['Close'+str(win+1)] = df2['Close'].shift(-(win)) #stepback i-1 times
  return df2

data = transdata(data3, 5)
ENR = pickle.load(open('model.pkl', 'rb'))
trainn = deepcopy(data.loc[0:10])
testt = deepcopy(data.loc[10:15])
trainnY = trainn["Close"+str(window+1)].values
del trainn["Close"+str(window+1)]
trainn.drop(columns=[])
trainnX = trainn.values.astype(np.float64)
testtY = testt["Close"+str(window+1)].values
del testt["Close"+str(window+1)]
testtX = testt.values.astype(np.float64) 
testp = ENR.predict(testtX)
print(mean_squared_error(testp, testtY))

In [ ]:
### Window Size is 5

### CNN
def split_sequence(sequence, n_steps):
  X, y = list(), list()
	for i in range(len(sequence)):
		# find the end of this pattern
		end_ix = i + n_steps
		# check if we are beyond the sequence
		if end_ix > len(sequence)-1:
			break
		# gather input and output parts of the pattern
		seq_x, seq_y = sequence[i:end_ix], sequence[end_ix]
		X.append(seq_x)
		y.append(seq_y)
	return array(X), array(y)
closing_univariate_data = list(data4["Close"])
n_steps = 5
X, y = split_sequence(closing_univariate_data, n_steps)
n_features = 1
X = X.reshape((X.shape[0], X.shape[1], n_features))
CNN = pickle.load(open('model.pkl', 'rb'))
X_train,y_train = X[0:10],y[0:10]
X_test,y_test = X[10:15],y[10:15]
testp = ENR.predict(X_test)
print(mean_squared_error(testp, Y_test))